### This tutorial demonstrates the binary analysis workflow of `BreathPy`.
Initially sample data (MCC-IMS measurements of breath after consuming either `menthol` or `citrus` candy, see material section of ([Weber et al. 2020][Weber2020]) for more information) is downloaded and split into a training and testing fraction - the test samples will later serve to test the created random forest classifier. The samples are normalized and denoised. Afterwards, several peak-detection methods are applied including the manuallly created `VisualnowLayer` contained in the sample data. Subsequently, peaks are aligned using the `ProbeClustering` approach and the features are reduced using `RemovePercentageFeatures` - which limits the reported features to the ones present in at least `percentage_threshold` of the minority class - in this case `citrus`. 
Features are then weighted using the `PerformanceMeasure`s `RANDOM_FOREST_CLASSIFICATION` and `FDR_CORRECTED_P_VALUE` - leaving 10 features each. 
Two decision trees and a `RandomForestClassifier` are trained. They serve as visual interpretation of the classifcation strategy based on each `PerformanceMeasure`.
After training, the `RandomForestClassifier` is used to predict the class labels of the test samples.
Finally, plots are created and saved in the `results/plots/` directory.

[Weber2020]: https://doi.org/10.3390/metabo10100393

In [ ]:
# suppress warnings for rendering as pdf
import warnings
warnings.filterwarnings('ignore')

# handle imports
from urllib.request import urlretrieve
from shutil import move as file_move
import numpy as np
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
import joblib

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

from breathpy.generate_sample_data import generate_train_test_sets, generate_train_test_set_helper
from breathpy.model.BreathCore import (MccImsAnalysis, MccImsMeasurement, PredictionModel,
                              construct_default_parameters,
                              construct_default_processing_evaluation_steps,
                              construct_custom_processing_evaluation_dict)
from breathpy.model.ProcessingMethods import FeatureReductionMethod, PerformanceMeasure, GCMSPeakDetectionMethod, GCMSAlignmentMethod
from breathpy.tools.tools import get_peax_binary_path

from breathpy.view.BreathVisualizations import ClusterPlot, HeatmapPlot, RocCurvePlot, BoxPlot, TreePlot, TimeSeriesPlot

In [ ]:
# download sample data and split into train and test fraction
url = 'https://github.com/philmaweb/BreathAnalysis.github.io/raw/master/data/full_candy.zip'
zip_dst = Path("data/full_candy.zip")
dst_dir = Path("data/full_candy/")
dst_dir.mkdir(parents=True, exist_ok=True)
urlretrieve(url, zip_dst)

# unzip archive into data subdirectory
with ZipFile(zip_dst, "r") as archive_handle:
    archive_handle.extractall(Path(dst_dir))

raw_dir = dst_dir
target_dir = Path("data/")

# split into train and test fraction - use 1/3 of samples for validation
_ = generate_train_test_sets(dir_full_set=raw_dir, root_target_dir=target_dir, cross_val_num=3, seed=42)

### Now we simplify and go step-by-step through the methods called by `breathpy.model.CoreTest.run_start_to_end_pipeline`

In [ ]:
%%capture
# capture print statements of comandline output 

# define default parameters and train / test directory
folder_name = file_prefix = 'train_full_candy'
plot_parameters, file_parameters = construct_default_parameters(file_prefix, folder_name, make_plots=True)

# create default parameters for preprocessing and evaluation
preprocessing_steps, evaluation_params_dict = construct_default_processing_evaluation_steps()

# define directory for training and test set
train_dir = Path("data/train_full_candy/")
test_dir = Path("data/test_full_candy/")

# get class label dict file from training set
train_class_label_dict_fn = MccImsAnalysis.guess_class_label_extension(train_dir)

# read in raw mcc-ims measurements of training set - based on class_label_dict
train_measurements = [MccImsMeasurement(fn) for fn in train_dir.glob("*ims.csv")]

In [ ]:
%%capture
# capture print statements of comandline output 

# setup analysis - need to get path of peax binary and get the visualnowlayer filename
visualnow_layer_path = Path("data/train_full_candy/candy_layer.xls")

# create output directory
if not Path(file_parameters['out_dir']).exists():
    Path(file_parameters['out_dir']).mkdir(parents=True, exist_ok=True)

# create analysis
ims_analysis = MccImsAnalysis(
    train_measurements, preprocessing_steps, performance_measure_parameters=evaluation_params_dict,
    class_label_file=train_class_label_dict_fn, dataset_name=folder_name, visualnow_layer_file=visualnow_layer_path,
    peax_binary_path=get_peax_binary_path())

### Preprocessing
Upon initialization with the raw training samples and associated class labels, the raw measurements are denoised and normalized using the default parameters.

Normalization and denoising methods are applied to reduce technical artifacts and instrumental noise and standardize the intensity range for measurements ([Engel et al. 2013][Engel2013]). Previous comprehensive studies of preprocessing methods for MCC-IMS data have highlighted, compared and discussed the best performing methods in detail in ([Smolinska et al. 2014][Smolinska2014], [Szymanska et al. 2016][Szymanska2016], [Horsch et al. 2017][Horsch2017]).

The default normalization methods are `INTENSITY_NORMALIZATION` and `BASELINE_CORRECTION`. They are both designed to reduce the influence of the reactant ion peak (RIP), which is a disturbance that appears in every MCC-IMS measurement at a characteristic inverse reduced mobility (IRM) position ([Smolinska et al. 2014][Smolinska2014]). As it influences the spectrum's intensity at the same retention time (RT), we try to remove its influence and make the intensity values more comparable. 
The `BASELINE_CORRECTION` method reduces the baseline of the spectra affected by the RIP by subtracting the 25%-quantile of the intensity (see ([Szymanska et al. 2016][Szymanska2016]) for more details), while the `INTENSITY_NORMALIZATION` method normalizes the intensity values based on the maximum intensity of the RIP.

Default denoising methods are `CROP_INVERSE_REDUCED_MOBILITY`, `DISCRETE_WAVELET_TRANSFORMATION`, `SAVITZKY_GOLAY_FILTER`, `MEDIAN_FILTER`, and `GAUSSIAN_FILTER`. `CROP_INVERSE_REDUCED_MOBILITY` removes the parts of all spectra that have IRM values below 0.4. 
`DISCRETE_WAVELET_TRANSFORMATION` compresses and reconstructs the spectra, for which we use the PyWavelets ([Lee et al. 2019][Lee2019]) implementation. The intuition behind this is that the compression will remove noise from the spectrum and only reconstruct the relevant signal.

`SAVITZKY_GOLAY_FILTER`, `MEDIAN_FILTER`, and `GAUSSIAN_FILTER` smooth the spectra using the neighboring intensity values, the weighted average intensity, or using a kernel method, which should remove noise, but will likely dampen the signal. They were implemented using SciPy ([Virtanen et al. 2020][Virtanen2020]).

Subsequently, the default peak-detection methods `PEAX` ([D'Addario et al. 2014][DAddario2014]
), `WATERSHED` ([Bunkowski et al. 2019][Bunkowski2019]), `VISUALNOWLAYER` ([Bödeker et al. 2008][Bödeker2008]), and `TOPHAT` ([Sternberg et al. 1986][Sternberg1986], [Virtanen et al. 2020][Virtanen2020]) are applied, resulting in the creation of a `PeakDetectionResult` for every sample and used peak-detection method. 

The elimination of manual peak-detection for MCC-IMS data and the development of automated methods have been extensively studied over the past decade ([Horsch et al. 2017][Horsch2017]), which allowed us to integrate successful tools and comparable approaches. The `VISUALNOW_LAYER` method extracts peaks based on provided coordinates, which are imported from an annotation file of the commercial Visual Now software (B&S Analytik, Dortmund, Germany) ([Bödeker et al. 2008][Bödeker2008]).


[Engel2013]: https://doi.org/10.1016/j.trac.2013.04.015
[Smolinska2014]: https://doi.org/10.1088/1752-7155/8/2/027105
[Szymanska2016]: https://doi.org/10.1039/C6AN01008C
[Horsch2017]: https://doi.org/10.1371/journal.pone.0184321
[Lee2019]: https://doi.org/10.21105/joss.01237
[Virtanen2020]: https://doi.org/10.1038/s41592-019-0686-2
[DAddario2014]: https://doi.org/10.1186/1471-2105-15-25
[Bunkowski2019]:https://pub.uni-bielefeld.de/publication/2517237
[Bödeker2008]: https://doi.org/10.1007/s12127-008-0012-7
[Sternberg1986]: http://www.sciencedirect.com/science/article/pii/0734189X86900046
[Weber2020]: https://doi.org/10.3390/metabo10100393

In [ ]:
%%capture

# run normalization, denoising and peak_detection for measurements using 6 cores
# for peak_detection we run [PEAX, WATERSHED, VISUALNOWLAYER, TOPHAT] methods defined in preprocessing_steps
# if one want to change default parameters, pass updated parameters for preprocessing_parameters
ims_analysis.preprocess_multicore(num_cores=6)

**Visualization**  

* Plot and show a chromatogram after normalization and denoising 

In [ ]:
# plot and show a preprocessed measurement
test_measurement = ims_analysis.measurements[0]
HeatmapPlot.FastIntensityMatrix(test_measurement, plot_parameters=plot_parameters, title=str(test_measurement))

Image(Path("results/plots/heatmaps/fast_train_full_candy_intentsity_plot_BD18_1408280834_ims.png"))

### Peak-Alignment
The implemented peak-alignment methods are `PROBE_CLUSTERING` and `DBSCAN` ([Ester et al. 1996][Ester1996]), which covered in detail in ([Weber et al. 2020][Weber2020]). The default method for peak-alignment is `PROBE_CLUSTERING`.

[Ester1996]: http://www.aaai.org/Papers/KDD/1996/KDD96-037.pdf
[Weber2020]: https://doi.org/10.3390/metabo10100393

In [ ]:
# align peak detection results
ims_analysis.align_peaks()

#### Visualization
* Plot and show clusters for each peak detection method
* Plot and show average chromatogram for each candy type - after normalization and denoising

In [ ]:
clusters = ClusterPlot.ClusterBasic(ims_analysis, plot_parameters=plot_parameters)
overlays = ClusterPlot.OverlayClasswiseAlignment(ims_analysis, plot_parameters=plot_parameters)

# get paths of the images
cluster_fn = Path(clusters[2][-1])
overlay_fn_citrus =Path("results/plots/overlay/")/overlays[2][-1]
overlay_fn_menthol =Path("results/plots/overlay/")/overlays[-2][-1]

# display images for the TOPHAT method
images = [cluster_fn, overlay_fn_citrus, overlay_fn_menthol]

for fn in images:
    display(Image(fn))

### Feature-Reduction
After peak-alignment, feature-reduction can be applied. In this process, the features are filtered using the `REMOVE_PERCENTAGE_FEATURES` method, which removes features that are lower than a `noise_threshold` or that are not represented in a certain percentage of samples.
The default `noise_threshold` is a permissive 0.0001, while the `percentage_threshold` requires that at least 50% of samples in any sample show the feature.

### Performance Evaluation
The result of feature-reduction are reduced sample-by-feature matrices.
Based on these matrices, a random forest classifier is trained and evaluated using three-fold cross-validation. During this process, features are ranked and selected for both performance measures `FDR_CORRECTED_P_VALUE` and `RANDOM_FOREST_CLASSIFICATION`.
False discovery rate corrected p-values are computed using the `FDR_CORRECTED_P_VALUE` method, which calculates p-values based on the Mann-Whitney-U test ([Mann and Whitney 1947][Mann1947]) and adjusted for multiple testing using the Benjamini-Hochberg procedure ([Benjamini and Hochberg 1995][Benjamini1995]). For our calculations, we use the implementations of statsmodels ([Seabold and Perktold 2010][Seabold2010]). 
In `RANDOM_FOREST_CLASSIFICATION`, the features are weighted based on the reported feature importance over the cross-validation loops. 

[Mann1947]: https://doi.org/10.1214/aoms/1177730491
[Benjamini1995]: https://doi.org/10.1111/j.2517-6161.1995.tb02031.x
[Seabold2010]: http://statsmodels.sourceforge.net

In [ ]:
# apply feature reduction
ims_analysis.reduce_features(ims_analysis.AVAILABLE_FEATURE_REDUCTION_METHODS)
# evaluate model performance using 3-fold cross-validation
ims_analysis.evaluate_performance()

### Export
In order to speed up future analysis or enable the analysis of intermediary results, we implemented an export and import functionality.
In this case, preprocessed measurements, peak detection results and feature_matrices are exported in the `.csv` format.

In [ ]:
%%capture
# export preprocessed files, peak detection results and feature_matrixes to csv into results directory
print(file_parameters['out_dir'])

# export preprocessed files, peak detection results and feature_matrixes to csv into results directory
ims_analysis.export_results_to_csv(file_parameters['out_dir'])

#### Visualize the analysis:
* Plot and show estimated model performance - ROC curve
* Plot and show best features superimposed for each candy type
* Plot and and show boxplot and time-series for each feature

In [ ]:
roc_plots = RocCurvePlot.ROCCurve(ims_analysis.analysis_result, plot_parameters=plot_parameters)
box_plots = BoxPlot.BoxPlotBestFeature(ims_analysis.analysis_result, plot_parameters=plot_parameters)

try:
    dt_plots = TreePlot.DecisionTrees(ims_analysis.analysis_result, plot_parameters=plot_parameters, limit_to_peak_detection_method_name="TOPHAT")
except FileNotFoundError as e:
    # might not both be installe - need system executable and python library
    print("Probably graphviz is not installed - install via `conda install graphviz python-graphviz`")
    raise(e)

ts_plots = TimeSeriesPlot.TimeSeriesFromAnalysis(ims_analysis, plot_parameters=plot_parameters, limit_to_pdmn=['TOPHAT'], limit_to_features=['Peak_0178','Peak_0231'])

roc_fn = roc_plots[2][-1]
box_plot_fn = box_plots[0][-1]
if dt_plots:
    dt_fn = Path(dt_plots[1][-1][-1])

ts_fn0 = ts_plots[1][0]
ts_fn1 = ts_plots[1][1]
    
# display images for the TOPHAT method
images = [roc_fn, box_plot_fn, dt_fn, ts_fn0, ts_fn1]

for fn in images:
    display(Image(fn))

### Validation and Prediction
A `PredictionModel` is created, which trains a random forest classifier on the best performing features for both performance measures and the complete training set.
Subsequently, it is initialized with the raw validation samples. Therefore, it needs to perform the same normalization, denoising, peak-detection, and alignment steps as performed in the training procedure before.

After this, it reduces the resulting feature matrices from the validation set to the ones present in the training matrices. Lastly, it predicts the classes for the validation samples and returns the classification results. These, in turn, are then used as the final results and lead to the evaluation of the validation performance.


In [ ]:
%%capture
# prediction - will use parameters and features from previous steps
predictionModel = PredictionModel(ims_analysis)

#  preparation - replace train_ with test_
#  otherwise can't find measurements - as the class labels don't match the measurement names
file_parameters['folder_path'] = file_parameters['folder_path'].replace("train_", "test_")
test_dir = file_parameters['folder_path']

test_result_dir = file_parameters['out_dir'].replace("train_", "test_")

test_measurements_fns = sorted(Path(test_dir).glob("*ims.csv"))
test_measurements = [MccImsMeasurement(fn) for fn in test_measurements_fns]

# predict - and run full preprocessing and alignment on test_measurements
prediction = predictionModel.predict(test_measurements)

### Validation of Prediction Model
For each of the included peak-detection methods the `PredictionModel` was applied to the samples in the validation set. Here the correct and predicted labels are compared, listing the correctly and misidentified class labels.

In [ ]:
test_labels_dict_fn = MccImsAnalysis.guess_class_label_extension(test_dir)
test_labels_dict = MccImsAnalysis.parse_class_labels(test_labels_dict_fn)
class_labels = np.unique([m.class_label for m in ims_analysis.measurements])
test_measurements_names = [path.name for path in test_measurements_fns]
for pdm, prediction_index in prediction.items():
    predicted_labels = {test_name: class_labels[p] for p, test_name in zip(prediction_index, test_measurements_names)}
    correct = dict()
    false = dict()
    for fn, predicted_label in predicted_labels.items():
        if predicted_label == test_labels_dict[fn]:
            correct[fn] = predicted_label
        else:
            false[fn] = predicted_label

    print(f"resulting_labels for {pdm.name} are: {predicted_labels}")
    print(f"Falsely classified: {false}\n")
    print(f"That's {len(correct.keys())} correct vs {len(false.keys())} false\n")

### Validation Set ROC-Curve
Finally a ROC-curve is plotted and shown for the performance of the prediction model on the validation set.

In [ ]:
# make a ROC curve for the predictions
roc_pred_plots = RocCurvePlot.ROCCurveFromPrediction(
        prediction_model = predictionModel, 
        prediction_class_label_dict = test_labels_dict, 
        plot_parameters=plot_parameters)
roc_pred_fn = roc_pred_plots[2][-1]
display(Image(roc_pred_fn))

### References

> P. Weber, J. K. Pauling, M. List, and J. Baumbach, “BALSAM—An Interactive Online Platformfor Breath Analysis, Visualization and Classification”, Metabolites 10, 393 (2020) [Weber et al. 2020][Weber2020].

> J. Engel, J. Gerretzen, E. Szymanska, J. J. Jansen, G. Downey, L. Blanchet, and L. M. Buydens,“Breaking with trends in pre-processing?”, TrAC Trends Anal. Chem.50, 96–106 (2013) [Engel et al. 2013][Engel2013].

> A. Smolinska, A.-C. Hauschild, R. R. R. Fijten, J. W. Dallinga, J. Baumbach, and F. J. vanSchooten, “Current breathomics—a review on data pre-processing techniques and machinelearning in metabolomics breath analysis”, J. Breath Res.8, 027105 (2014) [Smolinska et al. 2014][Smolinska2014].

> E. Szymanska, G. H. Tinnevelt, E. Brodrick, M. Williams, A. N. Davies, H.-J. van Manen, andL. M. Buydens, “Increasing conclusiveness of clinical breath analysis by improved baselinecorrection of multi capillary column – ion mobility spectrometry (MCC-IMS) data”, J. Pharm. Biomed. Analysis127, 170–175 (2016) [Szymanska et al. 2016][Szymanska2016].

> G. Lee, R. Gommers, F. Waselewski, K. Wohlfahrt, and A. O’Leary, “PyWavelets: A Pythonpackage for wavelet analysis”, J. Open Source Softw.4, 1237 (2019) [Lee et al. 2019][Lee2019].

> P. Virtanen, R. Gommers, T. E. Oliphant, M. Haberland, T. Reddy, D. Cournapeau, E. Burovski,P. Peterson, W. Weckesser, J. Bright, S. J. van der Walt, M. Brett, J. Wilson, K. J. Millman,N. Mayorov, A. R. J. Nelson, E. Jones, R. Kern, E. Larson, C. J. Carey, I. Polat, Y. Feng, E. W.Moore, J. VanderPlas, D. Laxalde, J. Perktold, R. Cimrman, I. Henriksen, E. A. Quintero,C. R. Harris, A. M. Archibald, A. H. Ribeiro, F. Pedregosa, and P. van Mulbregt, “SciPy1.0: fundamental algorithms for scientific computing in Python”, Nat. Methods17, 261–272(2020) [Virtanen et al. 2020][Virtanen2020].

> M. D’Addario, D. Kopczynski, J. Baumbach, and S. Rahmann, “A modular computationalframework for automated peak extraction from ion mobility spectra”, BMC Bioinforma.15,25 (2014) [D'Addario et al. 2014][DAddario2014].

> A. Bunkowski, “MCC-IMS data analysis using automated spectra processing and explorativevisualisation methods”, Ph.D. thesis, Bielefeld University (2011) [Bunkowski et al. 2019][Bunkowski2019].

> B. Bödeker, W. Vautz, and J. I. Baumbach, “Peak finding and referencing in MCC/IMS-data”, Int J Ion Mobil. Spectrom 11(2008) [Bödeker et al. 2008][Bödeker2008].

> S. Sternberg,  “Grayscale morphology”, Comput. vision,  graphics,  image processing pp.333–355 (1986) [Sternberg et al. 1986][Sternberg1986].

> S. Horsch,  D. Kopczynski,  E. Kuthe,  J. I. Baumbach,  S. Rahmann,  and J. Rahnenführer, “A  detailed  comparison  of  analysis  processes  for  MCC-IMS  data  in  disease  classifica-tion—Automated methods can replace manual peak annotations”, PLOS ONE 12, (2017) [Horsch et al. 2017][Horsch2017].

> H. B. Mann and D. R. Whitney, “On a Test of Whether one of Two Random Variables is Stochastically Larger than the Other”, The Annals Math. Stat. 18, 50–60 (1947) [Mann and Whitney 1947][Mann1947].

> Y.  Benjamini  and  Y.  Hochberg,  “Controlling the  False  Discovery  Rate:  A  Practical  and Powerful Approach to Multiple Testing”, J. Royal Stat. Soc. Ser. B (Methodological) 57,289–300 (1995) [Benjamini and Hochberg 1995][Benjamini1995].

> S. Seabold and J. Perktold, “Statsmodels: Econometric and Statistical Modeling with Python”, in PROC. OF THE 9th PYTHON IN SCIENCE CONF (2010) [Seabold and Perktold 2010][Seabold2010].


[Weber2020]: https://doi.org/10.3390/metabo10100393
[Smolinska2014]: https://doi.org/10.1088/1752-7155/8/2/027105
[Szymanska2016]: https://doi.org/10.1039/C6AN01008C
[Lee2019]: https://doi.org/10.21105/joss.01237
[Horsch2017]: https://doi.org/10.1371/journal.pone.0184321
[Engel2013]: https://doi.org/10.1016/j.trac.2013.04.015
[Virtanen2020]: https://doi.org/10.1038/s41592-019-0686-2
[DAddario2014]: https://doi.org/10.1186/1471-2105-15-25
[Bunkowski2019]:https://pub.uni-bielefeld.de/publication/2517237
[Bödeker2008]: https://doi.org/10.1007/s12127-008-0012-7
[Sternberg1986]: http://www.sciencedirect.com/science/article/pii/0734189X86900046
[Ester1996]: http://www.aaai.org/Papers/KDD/1996/KDD96-037.pdf
[Mann1947]: https://doi.org/10.1214/aoms/1177730491
[Benjamini1995]: https://doi.org/10.1111/j.2517-6161.1995.tb02031.x
[Seabold2010]: http://statsmodels.sourceforge.net